In [1]:
import os
import numpy as np
import cv2
import mediapipe as mp
import time
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import tensorflow as tf

In [ ]:
!pip install pilloW

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
!pip install opencv-python

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
!pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import sys  
sys.path.insert(0, './')
from pose_media import mediapipe_pose

In [3]:
DATA_PATH = "./DATA/" 
if not os.path.exists(DATA_PATH):
    os.mkdir(DATA_PATH) 

In [4]:
mp = mediapipe_pose()

In [5]:
cTime,pTime = 0,0

In [6]:

# Longitud de cada secuencia de frames
sequence_length = 30

# Ruta del directorio donde se encuentran los videos
path = "./DATASET/"

# Obtener la lista de acciones (carpetas) presentes en el directorio
actions = os.listdir(path)
actions = np.array(["Falling down","Headache","Nausea","Sit down", "Stand up","Walking", "running"])

# Recorrer cada acción
for action in actions:
    # Crear una carpeta para almacenar los keypoints de esa acción si no existe
    if not os.path.exists(DATA_PATH + action):
        os.mkdir(DATA_PATH + action)
    
    # Obtener la lista de videos presentes en la carpeta de esa acción
    video_list = os.listdir(path + action)
    
    # Recorrer cada video dentro de la acción
    for video_file in video_list:
        video_path = os.path.join(path, action, video_file)
        cap = cv2.VideoCapture(video_path)
        
        # Obtener la duración del video en frames
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        
        # Calcular los índices de los frames importantes
        important_frames = np.linspace(0, total_frames - 1, sequence_length, dtype=int)
        
        # Inicializar el detector de keypoints de MediaPipe
        with mp.mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
            # Crear una carpeta para almacenar los keypoints de ese video
            video_output_folder = os.path.join(DATA_PATH, action, video_file.split('.')[0])
            if not os.path.exists(video_output_folder):
                os.mkdir(video_output_folder)
            
            # Recorrer los índices de los frames importantes
            frame_num = 1
            for frame_index in important_frames:
                # Ir al frame importante
                cap.set(cv2.CAP_PROP_POS_FRAMES, frame_index)
                
                # Leer el frame
                ret, frame = cap.read()
                if not ret:
                    break
                
                try:
                    # Detectar keypoints en el frame utilizando MediaPipe
                    image, results = mp.mediapipe_detection(frame, holistic)
                except:
                    break
                
                # Dibujar los puntos clave detectados en el frame
                mp.draw_styled_landmarks(image, results)
                
                # Calcular y mostrar la tasa de FPS en la imagen
                cTime = time.time()
                fps = 1 / (cTime - pTime)
                pTime = cTime
                cv2.putText(image, "FPS:" + str(int(fps)), (10, 100), cv2.FONT_HERSHEY_PLAIN, 2, (255, 0, 190), 2, cv2.LINE_AA)
                
                # Mostrar el frame procesado en una ventana de OpenCV
                cv2.putText(image, 'Collecting frames for {} Video {}'.format(action, video_file), (5, 15), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 0, 255), 1, cv2.LINE_AA)
                cv2.imshow('OpenCV Feed', image)
                
                # Extraer los keypoints y guardarlos en un archivo numpy (.npy)
                keypoints = mp.extract_keypoints(results)
                npy_path = os.path.join(video_output_folder, str(frame_num))
                np.save(npy_path, keypoints)
                
                # Incrementar el contador de frames
                frame_num += 1
                
                # Salir del bucle si se presiona la tecla 'q'
                if cv2.waitKey(1) & 0xFF == ord("q"):
                    break
                    
            # Liberar la captura de video
            cap.release()

# Cerrar todas las ventanas de OpenCV
cv2.destroyAllWindows()


In [ ]:


import sys  
sys.path.insert(0, './')


DATA_PATH = "./DATA/" 
if not os.path.exists(DATA_PATH):
    os.mkdir(DATA_PATH)   


cTime,pTime = 0,0


sequence_length = 30
path = "./DATASET/"
actions = os.listdir(path)


DATA_PATH = os.path.join('DATA') 
actions = np.array(["Back pain","Chest pain","Falling down","Headache","Nausea","Neck pain","Sit down","Snezee-Cough","Staggering", "Stand up","Walking"])
no_sequences = 1776
sequence_length = 30

label_map = {label:num for num,label in enumerate(actions)}
print(label_map)

def load_data(actions,no_sequences,sequence_length):
    sequences1,labels = [],[]
    for action in actions:
        for sequence in range(no_sequences):
            window = []
            for frame_num in range(1,sequence_length+1):
                res = np.load(os.path.join(DATA_PATH,action,str(sequence),"{}.npy".format(frame_num)))
                window.append(res)
            sequences1.append(window)
            labels.append(label_map[action])
    return sequences1,labels

def cargar_datos(actions, sequences_length):
    frames_g, labels = [],[]
    for action in actions:
        




{'back_pain': 0, 'chest_pain': 1, 'falling_down': 2, 'headache': 3, 'neck_pain': 4, 'snezee': 5, 'staggering': 6}


In [16]:
DATA_PATH = os.path.join('DATA') 
CLASES = os.listdir( DATA_PATH)
label_map = {label:num for num,label in enumerate(CLASES)}
def load_dataset(DATA_PATH):
    CLASES = os.listdir( DATA_PATH)
    frames, labels = [], []
    for clase in CLASES:
        videos = os.listdir(os.path.join(DATA_PATH,clase))
        for video in videos:
            ventana = []
            for frame in range(1,sequence_length+1):
                aq = np.load(os.path.join(DATA_PATH,clase, video, "{}.npy".format(frame)))
                ventana.append(aq)
            frames.append(ventana)
            labels.append(label_map[clase])
    return frames, labels 

print(label_map)


{'Falling down': 0, 'Sit down': 1, 'Stand up': 2, 'still sitting': 3, 'Walking': 4}


In [17]:

%time sequences,labels = load_dataset(DATA_PATH)

CPU times: total: 15.8 s
Wall time: 18.1 s


In [18]:
X = np.array(sequences)
X.shape

(1535, 30, 132)

In [19]:
y = to_categorical(labels).astype(int)
print(len(y))

1535


In [20]:
with open('X4.npy', 'wb') as f:
    np.save(f, X)
with open('y4.npy', 'wb') as f:
    np.save(f, y)

In [21]:
os.listdir('DATA')

['Falling down', 'Sit down', 'Stand up', 'still sitting', 'Walking']